In [1]:
# import libraries
import pandas as pd
import plotly.graph_objects as go

In [ ]:
# load data
data = pd.read_csv("shops_table.csv", sep=";")

In [3]:
# preview data
data.head()

,Magasin,Type,Enseigne,Adresse Google,Département,Types d'algues,Marques,map_address,map_postcode,map_city,map_website,map_latitude,map_longitude,map_openclose
0,Biocoop l'Epine Vinette,Magasin bio,Biocoop,"Zone commerciale Sud, 230 Rue des Couteliers, ...",05 Hautes-Alpes,"Algues déshydratées,Algues transformées",Aoré,230 Rue des Couteliers,5100,Briançon,https://www.biocoop-epinevinette.com/,44.884755,6.626144,open
1,Biocoop Le Grenier Daudet,Magasin bio,Biocoop,"3 Rue Alphonse Daudet, 05000 Gap",05 Hautes-Alpes,NaN,NaN,3 Rue Alphonse Daudet,5000,Gap,https://www.biocooplegrenier.fr/,44.549183,6.065531,open
2,Biocoop Méditerranée,Magasin bio,Biocoop,"609 Rte de la Roquette, 06250 Mougins",06 Alpes-Maritimes,"Algues déshydratées,Algues transformées",Aoré,609 Route de la Roquette,6250,Mougins,https://mougins.biocoop.net/,43.612203,6.971844,open
3,Bio c' Bon Marseille Blancarde,Magasin bio,Bio c' Bon,"17 Bd de la Blancarde, 13004 Marseille",13 Bouches-du-Rhône,NaN,NaN,17 Boulevard de la Blancarde,13004,Marseille,https://www.bio-c-bon.eu/magasin/bio-c-bon-mar...,43.302776,5.397936,open
4,Bio c' Bon Marseille Cours Julien,Magasin bio,Bio c' Bon,"85 Cr Julien, 13006 Marseille",13 Bouches-du-Rhône,NaN,NaN,85 Cours Julien,13006,Marseille,https://www.bio-c-bon.eu/magasin/bio-c-bon-mar...,43.292653,5.384100,open


In [4]:
# replace nans in shop type
mask_nan = data["Type"].isnull()
data.loc[mask_nan, "Type"] = "Autre"

# assign some shop types to other type of shops
data.loc[data["Type"] == "Épicerie asiatique", "Type"] = "Autre"
data.loc[data["Type"] == "Poissonnerie", "Type"] = "Autre"
data.loc[data["Type"] == "Marché", "Type"] = "Autre"

In [ ]:
fig=go.Figure()
config = {'displayModeBar': True}

# set shop types and corresponding colors
shoptypes = ["Magasin bio", "Épicerie fine", "Grande ou moyenne surface", 
    "Autre", "Magasin de producteurs"]
colors = ["#93C9F9", "#007DED", "#C63074", "#1C9390", "#F9A115"]

# loop through shop types
for i in range(0,len(shoptypes)):
    # set type of shop
    shoptype= shoptypes[i]
    mask_shop = data["Type"] == shoptype
    # get useful intel
    longitude = data.loc[mask_shop, "map_longitude"]
    latitude = data.loc[mask_shop, "map_latitude"]
    shop_name = data.loc[mask_shop, "Magasin"]
    shop_data = data.loc[mask_shop, ["Type", "Magasin", "map_address",
        "map_postcode", "map_city", "map_website"]]
    # plot shop type on map
    fig.add_trace(
        go.Scattermap(
            lon = longitude,
            lat = latitude,
            name=shoptype,
            mode='markers',
            marker=dict(
                size=10,
                color=colors[i],
                ),
            customdata=shop_data,
            hovertemplate="""<a style="color:#FDF2EE" href="%{customdata[5]}">""" + \
                """<b style="font-family:'Parkinsans'; font-weight: 600">%{customdata[1]}</b><br>""" + \
                """<span style="font-family:'Montserrat'">%{customdata[2]}<br>""" + \
                """%{customdata[3]} %{customdata[4]}<br></span></a>""" + \
                """<extra><b style="font-family:'Parkinsans'">%{customdata[0]}</b></extra>""",
            cluster=dict(
                enabled=True,
                color=colors[i],
                opacity=1,
                size=15,
                maxzoom=9
                )
        ))

fig.update_layout(
    hovermode="closest",
    hoverlabel=dict(
        font_size=14,
        bordercolor="white",
    ),
    hoverdistance=100,
    map=dict(
        bearing=0,
        center=dict(
            lat=47,
            lon=2
        ),
        zoom=4.5,
        style="basic"
    ),
    legend=dict(
        orientation="h",
        font_family="Montserrat",
        font_color="#113972",
        bgcolor="white",
        entrywidth=225,
        yanchor="top",
        y=0.0,
        xanchor="center",
        x=0.5
    ),
    modebar=dict(
        orientation="v",
    ),
    margin=dict(
        l=30,
        r=30,
        t=0,
        b=20
    ),
    height=600,
    width=600
)

fig.show(config=config)

In [6]:
fig.write_html("shops_map.html", include_plotlyjs="cdn")